In [1]:
# install the library
!pip install numpy

  Using cached numpy-1.26.3-cp310-cp310-win_amd64.whl (15.8 MB)



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached regex-2023.12.25-cp310-cp310-win_amd64.whl (269 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl (300.9 MB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
     ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
     -------------------------------------  133.1/133.7 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 133.7/133.7 kB 4.0 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
     ----------------- -------------------- 194.6/413.4 kB 3.9 MB/s eta 0:00:01
     -------------------------------------- 413.4/413.4 kB 4.3 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl (441 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached tensorboar

## Import Library

In [5]:
import random
import json
import pickle
import numpy as np


import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data 

In [10]:
import json

# Fungsi untuk membaca data intents dari file JSON
def load_intents_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        intents_data = json.load(file)
    return intents_data

# Menentukan path file intents.json
intents_file_path = 'intents.json'

# Memuat data intents dari file JSON
intents = load_intents_from_json(intents_file_path) 

In [11]:
print(json.dumps(intents, indent=2)) 

{
  "intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hello",
        "Hi",
        "Halo",
        "Hai",
        "Magandang umaga",
        "Kamusta",
        "Magandang hapon",
        "Magandang gabi"
      ],
      "responses": [
        "Hello! Welcome to FashionFusion. I can help you find the perfect outfit. Is there anything specific you're looking for?"
      ]
    },
    {
      "tag": "StockItems",
      "patterns": [
        "I want to know what stock of clothes is available right now",
        "what are the popular types of clothing currently?",
        "I am looking for popular clothing now, what do you recommend?",
        "can you give me a list of best-selling products?"
      ],
      "responses": [
        "We have a variety of trendy clothes available right now, including floral dresses, oversized shirts, and denim jackets. To view the catalog please visit FashionFusion.com.",
        "In our collection, you will find a variety of interesting 

## Preprocessing 

In [20]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())     

In [22]:
#tokenization and corpus formation
words = []
classes = []
documents = []
ignore_latters = "?!.,/'"

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_latters]
words = [word.lower() for word in words if word not in ignore_latters]
words = sorted(set(words))
print("Word list after preprocessing:")
print(words)

print("Number of tokens generated:", len(words)) 
print("Number of tags generated:", len(classes))       

Word list after preprocessing:
['a', 'affordable', 'all', 'am', 'are', 'available', 'average', 'best-selling', 'can', 'check', 'clothes', 'clothing', 'currently', 'do', 'expensive', 'for', 'gabi', 'give', 'hai', 'halo', 'hapon', 'hello', 'here', 'hi', 'how', 'i', 'information', 'is', 'it', 'kamusta', 'know', 'list', 'looking', 'magandang', 'me', 'much', 'need', 'now', 'of', 'okay', 'online', 'order', 'ordering', 'popular', 'price', 'procedure', 'product', 'range', 'recommend', 'right', 'sold', 'stock', 'thank', 'that', 'the', 'there', 'to', 'type', 'umaga', 'want', 'website', 'what', 'will', 'you']
Number of tokens generated: 64
Number of tags generated: 5


In [23]:
# creation and saving of pixel models using the pickle module.
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb')) 

## Training 

In [24]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

rmsprop = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, validation_split=0.2, verbose=1)
model.save('cahtbotmodel4.model', hist)
print("The text preprocessing process is complete.") 


Epoch 1/100


4/4 [==============================] - 2s 97ms/step - loss: 1.6039 - accuracy: 0.2353 - val_loss: 1.7018 - val_accuracy: 0.2000
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 1.4681 - accuracy: 0.5882 - val_loss: 1.6415 - val_accuracy: 0.4000
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 1.2336 - accuracy: 0.4706 - val_loss: 1.3909 - val_accuracy: 0.6000
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 0.7714 - accuracy: 0.7647 - val_loss: 1.2511 - val_accuracy: 0.6000
Epoch 5/100
4/4 [==============================] - 0s 16ms/step - loss: 0.7014 - accuracy: 0.7059 - val_loss: 1.3957 - val_accuracy: 0.6000
Epoch 6/100
4/4 [==============================] - 0s 15ms/step - loss: 0.5312 - accuracy: 0.8235 - val_loss: 1.3599 - val_accuracy: 0.6000
Epoch 7/100
4/4 [==============================] - 0s 16ms/step - loss: 0.6116 - accuracy: 0.7647 - val_loss: 1.5921 - val_accuracy: 0.6000
Epoch 8/100
4/4 [

INFO:tensorflow:Assets written to: cahtbotmodel4.model\assets


The text preprocessing process is complete.
